In [1]:
!pip install flask flask_cors nltk keras


In [3]:
# libraries
import random
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")


# init file
words = []
classes = []
documents = []
ignore_words = ["?", "!"]
data_file = open("intents.json").read()
intents = json.loads(data_file)

# words
for intent in intents["intents"]:
    for pattern in intent["patterns"]:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent["tag"]))

        # adding classes to our class list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

# lemmatizer
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)


pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)

# Separate bag-of-words representations and output labels
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]

# Convert to NumPy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)
print("Training data created")

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()


sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5", hist)
print("model created")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


146 documents
93 classes ['agriculture_sector', 'author', 'british_rule_duration', 'cba_strike_endorsement_proportion', 'certified_collective_bargaining_agents', 'child_employment_age_limit', 'collective_bargaining', 'collective_bargaining_agreement_termination', 'collective_bargaining_introduction', 'conclusion', 'constitution', 'constitutional_article_association_unions', 'current_labor_laws_count', 'definitions', 'disputes_strikes', 'drafting_labor_legislation_ministry', 'employer_definition_industrial_relations', 'employment_contract', 'enacting_date', 'enforcement_challenges', 'equality', 'exempt_sectors_from_labor_laws', 'factories_act_1934', 'first_labor_legislation_british_india', 'freedom_association', 'gender_discrimination_prohibition', 'group_litigation_damages_level', 'illegal_strikes_punishment', 'individual_grievance_limitation_period', 'industrial_disputes_act_1947', 'industrial_disputes_act_1947_date', 'industrial_disputes_act_1947_definitions', 'industrial_disputes_ac

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Install pyngrok
!pip install pyngrok

from flask import Flask, request, jsonify
from flask_cors import CORS
import nltk
import numpy as np
import random
from keras.models import load_model
import pickle
import json
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)

# Load chatbot model and necessary data
chatbot_model = load_model("chatbot_model.h5")
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))
lemmatizer = nltk.stem.WordNetLemmatizer()

# Load intents JSON data
data_file = open("intents.json").read()
intents = json.loads(data_file)

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            responses = i["responses"]
            result = random.choice(responses)
            break
    return result

@app.route('/predict', methods=['POST'])
def predict():
    print('Received POST request at /predict endpoint')
    try:
        data = request.get_json()
        user_input = data['userInput']

        # Use the chatbot model for predictions
        chatbot_response = getResponse(predict_class(user_input, chatbot_model), intents)

        return jsonify({'response': chatbot_response})

    except Exception as e:
        return jsonify({'error': str(e)})

@app.route('/', methods=['GET'])
def root():
    print('Received GET request at root ("/") endpoint')
    return jsonify({'message': 'Welcome to the Flask server!'})

# Disable Flask's development server in production
if __name__ == '__main__':
    # Set up the ngrok tunnel with your authtoken
    ngrok.set_auth_token("2alSlyCj2f8uiD2GtlslJLep0UK_2gy915SobC3EM515e8o6k")
    public_url = ngrok.connect(addr="5000", proto="http", bind_tls=True)

    print(' * Tunnel URL:', public_url)
    app.run(port=5000, debug=False)

 * Tunnel URL: NgrokTunnel: "https://577a-34-23-141-141.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Received POST request at /predict endpoint
1/1 [==============================] - 0s 101ms/step


INFO:werkzeug:127.0.0.1 - - [19/Jan/2024 19:16:05] "POST /predict HTTP/1.1" 200 -


In [ ]:
!curl -X POST -H "Content-Type: application/json" -d '{"userInput": "Hello"}' https://209b-35-245-146-152.ngrok-free.app/predict


<!DOCTYPE html>
<html lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Text.woff" as="fon

In [ ]:
!pip install pyngrok

